In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px

mapbox_token = 'pk.eyJ1IjoiYW5kcmVhc29zdGVkIiwiYSI6ImNsbmJxMjFndDA4dm8ybXJrMzhia2NqdnoifQ.fXDNIJ1LelhA1ypNiaJE9w'

df = pd.read_csv('Data/homicide_data.csv')
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('state')

In [3]:
df = pd.read_csv('Data/homicide_data.csv')
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('city')

In [4]:
filtered_df = df[df['state'] == 'DC']
np.min(filtered_df.lon), np.max(filtered_df.lon)

(-77.095084, -76.9119737)

In [16]:
def make_map(city):
    if city == 'all cities':
        filtered_df = df
    else: 
        filtered_df = df[df['city'] == city]
  
    fig = px.scatter_mapbox(
        filtered_df,
        lat="lat",
        lon="lon",
        hover_name="victim_full_name",
        hover_data=["victim_race", "victim_age", "victim_sex"],
        color = "disposition",
        height=700
    )
    
    if city is not None:
        title = f'Murders in {city}'
    else:
        title = ''

    
    fig.update_layout(mapbox_style="light", mapbox_accesstoken=mapbox_token, title = title)
    fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
    fig.update_layout(clickmode='event+select',
                      hovermode='closest')


    
    return fig

app = dash.Dash(__name__)


cityoptions=[{'label': 'Select all', 'value': 'all cities'}]+[{'label': city, 'value': city} for city in sorted(df['city'].unique())]
# Define layout
app.layout = html.Div([
    dcc.Dropdown(
        id='state-dropdown',
        options=cityoptions,
        #value=df['city'].unique()[0],  # Set the default value
        value = None,
        placeholder = "Select city",
        style={'width': '200px'}
    ),
    dcc.Graph(id='murder-map')
])

# Define callback to update the map when dropdown selection changes
@app.callback(
    Output('murder-map', 'figure'),
    [Input('state-dropdown', 'value')]
)
def update_map(selected_state):
    fig = make_map(selected_state)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

